<h1><center>Laboratorio 9: Optimización de modelos 💯</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### Cuerpo Docente:

- Profesores: Ignacio Meza, Sebastián Tinoco
- Auxiliares: Catherine Benavides y Consuelo Rojas
- Ayudante: Nicolás Ojeda, Eduardo Moya

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Pía Antiquera.
- Evelyn Silva.


### Temas a tratar

- Predicción de demanda usando `xgboost`
- Búsqueda del modelo óptimo de clasificación usando `optuna`
- Uso de pipelines.

### Reglas:

- **Grupos de 2 personas**
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias.
- Pueden usar cualquer matrial del curso que estimen conveniente.
- Código que no se pueda ejecutar, no será revisado.

### Objetivos principales del laboratorio

- Optimizar modelos usando `optuna`
- Recurrir a técnicas de *prunning*
- Forzar el aprendizaje de relaciones entre variables mediante *constraints*
- Fijar un pipeline con un modelo base que luego se irá optimizando.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `pandas`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre DataFrames.

### **Link de repositorio de GitHub:** `https://github.com/piaantiquera/Courses-MDS7202.git`

# Importamos librerias útiles

In [1]:
!pip install -qq xgboost optuna

In [2]:
!pip install optuna-integration

# El emprendimiento de Fiu

Tras liderar de manera exitosa la implementación de un proyecto de ciencia de datos para caracterizar los datos generados en Santiago 2023, el misterioso corpóreo **Fiu** se anima y decide levantar su propio negocio de consultoría en machine learning. Tras varias e intensas negociaciones, Fiu logra encontrar su *primera chamba*: predecir la demanda (cantidad de venta) de una famosa productora de bebidas de calibre mundial. Como usted tuvo un rendimiento sobresaliente en el proyecto de caracterización de datos, Fiu lo contrata como *data scientist* de su emprendimiento.

Para este laboratorio deben trabajar con los datos `sales.csv` subidos a u-cursos, el cual contiene una muestra de ventas de la empresa para diferentes productos en un determinado tiempo.

Para comenzar, cargue el dataset señalado y visualice a través de un `.head` los atributos que posee el dataset.

<i><p align="center">Fiu siendo felicitado por su excelente desempeño en el proyecto de caracterización de datos</p></i>
<p align="center">
  <img src="https://media-front.elmostrador.cl/2023/09/A_UNO_1506411_2440e.jpg">
</p>

In [3]:
### Importación de librerias

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.dummy import DummyRegressor
from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import ColumnTransformer
# Se importan distintos métodos para escalar ya que se prueban todos.
from sklearn.preprocessing import OneHotEncoder, PowerTransformer, StandardScaler, MinMaxScaler
from sklearn.metrics import mean_absolute_error
from xgboost import XGBRegressor
import optuna 
import pandas as pd
import numpy as np
from datetime import datetime
optuna.logging.set_verbosity(optuna.logging.WARNING)
from optuna.integration import XGBoostPruningCallback 
from xgboost import XGBRegressor
from optuna.visualization import plot_optimization_history
from optuna.visualization import plot_parallel_coordinate
from optuna.visualization import plot_param_importances
import joblib
import pickle
import warnings
warnings.filterwarnings("ignore")

In [4]:
df = pd.read_csv('sales.csv')
df['date'] = pd.to_datetime(df['date'])

print('Min quantity:', df['quantity'].min())
print('Max quantity:', df['quantity'].max())
print('Mean quantity:', df['quantity'].mean())

Min quantity: 2953
Max quantity: 145287
Mean quantity: 29408.428379828325


## 1 Generando un Baseline (0.5 puntos)

<p align="center">
  <img src="https://media.tenor.com/O-lan6TkadUAAAAC/what-i-wnna-do-after-a-baseline.gif">
</p>

Antes de entrenar un algoritmo, usted recuerda los apuntes de su magíster en ciencia de datos y recuerda que debe seguir una serie de *buenas prácticas* para entrenar correcta y debidamente su modelo. Después de un par de vueltas, llega a las siguientes tareas:

1. Separe los datos en conjuntos de train (70%), validation (20%) y test (10%). Fije una semilla para controlar la aleatoriedad.
2. Implemente un `FunctionTransformer` para extraer el día, mes y año de la variable `date`. Guarde estas variables en el formato categorical de pandas.
3. Implemente un `ColumnTransformer` para procesar de manera adecuada los datos numéricos y categóricos. Use `OneHotEncoder` para las variables categóricas.
4. Guarde los pasos anteriores en un `Pipeline`, dejando como último paso el regresor `DummyRegressor` para generar predicciones en base a promedios.
5. Entrene el pipeline anterior y reporte la métrica `mean_absolute_error` sobre los datos de validación. ¿Cómo se interpreta esta métrica para el contexto del negocio?
6. Finalmente, vuelva a entrenar el `Pipeline` pero esta vez usando `XGBRegressor` como modelo **utilizando los parámetros por default**. ¿Cómo cambia el MAE al implementar este algoritmo? ¿Es mejor o peor que el `DummyRegressor`?
7. Guarde ambos modelos en un archivo .pkl (uno cada uno)

In [5]:
### 1. Particion del conjunto de datos ---------------------------------------------------------------------------------------------------------------------------------------------------
train, test = train_test_split(df, test_size=0.1, random_state=42)
train, val = train_test_split(train, test_size=0.2, random_state=42)
# Subconjuntos X
X_train = train.drop(['quantity','id'], axis=1)
X_val = val.drop(['quantity','id'], axis=1)
X_test = test.drop(['quantity','id'], axis=1)
# Subconjuntos Y
Y_train = train['quantity']
Y_val = val['quantity']
Y_test = test['quantity']


### 2. Definición functionTransformer -----------------------------------------------------------------------------------------------------------------------------------------------------
def transform_date(df):
    # Obtener columna date con formato de fecha
    df['date'] = pd.to_datetime(df['date'])
    # Extraer el año de la fecha como categorico.
    df['year'] = df['date'].dt.year.astype('category')
    # Extrar el mes de la fecha como categorico.
    df['month'] = df['date'].dt.month.astype('category')
    # Extraer el dia de la fecha como categorico.
    df['day'] = df['date'].dt.day.astype('category')
    # Eliminar la columna date por redundancia
    df_mod = df.drop('date', axis=1)
    
    return df_mod

### 3. Definición de ColumnTransformer ---------------------------------------------------------------------------------------------------------------------------------------------------
def get_categorical(df):
    categorical_cols = df.select_dtypes(include=['object', 'category']).columns.tolist()
    return categorical_cols

def get_numerical(df):
    numerical_cols = df.select_dtypes(include=['float64', 'int64']).columns.tolist()
    numerical_cols.remove('id')
    numerical_cols.remove('quantity')
    return numerical_cols

categorical_cols = get_categorical(train)
numerical_cols = get_numerical(train)

preprocessor = ColumnTransformer([
    ('Numerical_cols', PowerTransformer(), numerical_cols),
    ('Categorical_cols', OneHotEncoder(sparse_output=True), categorical_cols)
])


### 4. Implementación de Pipeline con DummyRegressor ------------------------------------------------------------------------------------------------------------------------------------

# Creacion del pipeline
pipeline_dummyregressor = Pipeline([
    # Implementación de FuncionTransformer en columna 'date'
        ('Transform_date', FunctionTransformer(transform_date)),
    # Implementación de ColumnTransformer en columnas numericas y categoricas
        ('Preprocessor', preprocessor),
    # Implementación de FunctionTransformer para transformar el output a df
        ('Convert_df', FunctionTransformer(
        lambda X: pd.DataFrame(X, columns=numerical_cols + preprocessor.transformers_[1][1].get_feature_names_out().tolist()))),
    # Implementación de DummyRegresor
        ('DummyRegressor', DummyRegressor(strategy='mean'))
        ])

### 5. Entrenamiento del pipeline y obtención de MAE desde validacion --------------------------------------------------------------------------------------------------------------------

# Entrenamiento del pipeline
pipeline_dummyregressor.fit(X = X_train, y=Y_train)

# Prediccion en particion de validacion
Y_val_preds = pipeline_dummyregressor.predict(X_val)
# Obtencion de métrica MAE a partir de las predicciones en particion de validacion
MAE_val_dummyregressor = mean_absolute_error(Y_val, Y_val_preds)
print(f'MAE en particion de validación (DummyRegressor): {MAE_val_dummyregressor}')

### 6. Implementacion de Pipeline con XGBRegressor  --------------------------------------------------------------------------------------------------------------------------------------

# Creacion del pipeline
pipeline_xgbregressor = Pipeline([
    # Implementación de FuncionTransformer en columna 'date'
        ('Transform_date', FunctionTransformer(transform_date)),
    # Implementación de ColumnTransformer en columnas numericas y categoricas
        ('Preprocessor', preprocessor),
    #  Implementación de FunctionTransformer para transformar el output a df
        ('Convert_df', FunctionTransformer(
        lambda X: pd.DataFrame(X, columns=numerical_cols + preprocessor.transformers_[1][1].get_feature_names_out().tolist()))),
    # Implementación de XGBRegressor
        ('XGBRegressor', XGBRegressor())
        ])

# Entrenamiento del pipeline
pipeline_xgbregressor.fit(X = X_train, y=Y_train)
# Prediccion en particion de validacion
Y_val_preds = pipeline_xgbregressor.predict(X_val)
# Obtencion de métrica MAE a partir de las predicciones en particion de validacion
MAE_val_xgbregressor = mean_absolute_error(Y_val, Y_val_preds)
print(f'MAE en particion de validación (XGBRegressor): {MAE_val_xgbregressor}')

### 7. Guardar modelos en pkl -----------------------------------------------------------------------------------------------------------------------------------------------------------
# No los guardamos porque no los vamos a usar despues
#joblib.dump(pipeline_dummyregressor, 'dummyregressor.pkl')
#joblib.dump(pipeline_xgbregressor, 'xgbregressor.pkl')

MAE en particion de validación (DummyRegressor): 13576.56699051175
MAE en particion de validación (XGBRegressor): 7012.852410624883


5. A partir de la implementación del pipeline con DummyRegressor, notamos que se tiene un MAE de 13576. El MAE corresponde a las diferencias absolutas entre las predicciiones del modelo de la demanda y el valor real de la demanda, por lo tanto nos dice, en promedio, cuanto se desvia o "que tan lejos" está la estimación de la demanda respecto a su valor real. 

6. A partir de la implementación del pipeline con XGBRegressor, notamos que se tiene un MAE de 7012, es decir, tuvo una mejora significativa en la precisión del modelo reflejada por la reducción del valor del MAE reportado. Lo anterior sugiere que el XGBRegressor presenta un mejor rendimiento general dado que es mas eficiente al momento de predecir la demanda dado que es capaz de capturar patrones importantes en los datos que el DummyRegressor no puede dada su naturaleza (de predecir la media de los valores de entrenamiento).


## 2. Forzando relaciones entre parámetros con XGBoost (1.0 puntos)

<p align="center">
  <img src="https://64.media.tumblr.com/14cc45f9610a6ee341a45fd0d68f4dde/20d11b36022bca7b-bf/s640x960/67ab1db12ff73a530f649ac455c000945d99c0d6.gif">
</p>

Un colega aficionado a la economía le *sopla* que la demanda guarda una relación inversa con el precio del producto. Motivado para impresionar al querido corpóreo, se propone hacer uso de esta información para mejorar su modelo realizando las siguientes tareas:

1. Vuelva a entrenar el `Pipeline`, pero esta vez forzando una relación monótona negativa entre el precio y la cantidad. Para aplicar esta restricción apóyese en la siguiente <a href = https://xgboost.readthedocs.io/en/stable/tutorials/monotonic.html>documentación</a>. Hint: Para implementar el constraint se le sugiere hacerlo especificando el nombre de la variable. De ser así, probablemente le sea útil **mantener el formato de pandas** antes del step de entrenamiento.

2. Luego, vuelva a reportar el `MAE` sobre el conjunto de validación.

3. ¿Cómo cambia el error al incluir esta relación? ¿Tenía razón su amigo?




In [6]:
# 1. Creacion del pipeline y obtención de MAE desde validacion ------------------------------------------------------------------------------------------------------------------------------------------------------------
pipeline_relation = Pipeline([
    # Implementación de FuncionTransformer en columna 'date'
        ('Transform_date', FunctionTransformer(transform_date)),
    # Implementación de ColumnTransformer en columnas numericas y categoricas
        ('Preprocessor', preprocessor),
    # Implementación de FunctionTransformer para transformar el output a df
        ('Convert_df', FunctionTransformer(
        lambda X: pd.DataFrame(X, columns=numerical_cols + preprocessor.transformers_[1][1].get_feature_names_out().tolist()))),
    # Implementación de XGBRegressor incluyendo la relacion monotona negativa con el precio
        ('XGBRegressor', XGBRegressor(interaction_constraints=[{'price':-1}]))
        ])

# Entrenamiento del pipeline
pipeline_relation.fit(X = X_train, y=Y_train)
# Prediccion en particion de validacion
Y_val_preds = pipeline_relation.predict(X_val)
# Obtencion de métrica MAE a partir de las predicciones en particion de validacion
MAE_val_xgbregressor_ct = mean_absolute_error(Y_val, Y_val_preds)
print(f'MAE en particion de validación (XGBRegressor con relación): {MAE_val_xgbregressor_ct}')

# No lo guardamos porque no lo usaremos despues
xgbregressor_ct = XGBRegressor(interaction_constraints=[{'price':-1}])
joblib.dump(xgbregressor_ct, 'pipeline_xgbregressor_ct.pkl')


MAE en particion de validación (XGBRegressor con relación): 7367.636681865118


['pipeline_xgbregressor_ct.pkl']

3.  A partir de la implementación del pipeline con XGBRegressor forzando una restricción de relación monótona negativa entre el precio y la cantidad, notamos que se tienee un MAE de 7367, es decir, en comparación con el pipeline con XGBRegressor que no tiene esta restricción, no se presenta una mejora en el rendimiento general del modelo reflejado en el aumento del MAE. Por lo tanto, pareciera que la relación monótona negativa entre precio y cantidad no es del todo cierta pues de ser así, deberia presentarse una disminución del MAE en comparación con el mismo modelo sin la restricción de relación.

## 3. Optimización de Hiperparámetros con Optuna (2.0 puntos)

<p align="center">
  <img src="https://media.tenor.com/fmNdyGN4z5kAAAAi/hacking-lucy.gif">
</p>

Luego de presentarle sus resultados, Fiu le pregunta si es posible mejorar *aun más* su modelo. En particular, le comenta de la optimización de hiperparámetros con metodologías bayesianas a través del paquete `optuna`. Como usted es un aficionado al entrenamiento de modelos de ML, se propone implementar la descabellada idea de su jefe.

A partir de la mejor configuración obtenida en la sección anterior, utilice `optuna` para optimizar sus hiperparámetros. En particular, se le pide:

- Fijar una semilla en las instancias necesarias para garantizar la reproducibilidad de resultados
- Utilice `TPESampler` como método de muestreo
- De `XGBRegressor`, optimice los siguientes hiperparámetros:
    - `learning_rate` buscando valores flotantes en el rango (0.001, 0.1)
    - `n_estimators` buscando valores enteros en el rango (50, 1000)
    - `max_depth` buscando valores enteros en el rango (3, 10)
    - `max_leaves` buscando valores enteros en el rango (0, 100)
    - `min_child_weight` buscando valores enteros en el rango (1, 5)
    - `reg_alpha` buscando valores flotantes en el rango (0, 1)
    - `reg_lambda` buscando valores flotantes en el rango (0, 1)
- De `OneHotEncoder`, optimice el hiperparámetro `min_frequency` buscando el mejor valor flotante en el rango (0.0, 1.0)
- Explique cada hiperparámetro y su rol en el modelo. ¿Hacen sentido los rangos de optimización indicados?
- Fije el tiempo de entrenamiento a 5 minutos
- Reportar el número de *trials*, el `MAE` y los mejores hiperparámetros encontrados. ¿Cómo cambian sus resultados con respecto a la sección anterior? ¿A qué se puede deber esto?
- Guardar su modelo en un archivo .pkl

In [7]:
# Funcion para optimizar parametros del modelo --------------------------------------------------------------------------------------------------------------------------------------------
def optimize_function(trial):
    # Definición de rangos de optimización por hiperparámetro para XGBRegressor
    params_xgb = {
        "learning_rate": trial.suggest_float("learning_rate", 0.001, 0.1),
        "n_estimators": trial.suggest_int("n_estimators", 50, 1000),
        "max_depth": trial.suggest_int("max_depth", 3, 10),
        "max_leaves": trial.suggest_int("max_leaves", 0, 100),
        "min_child_weight": trial.suggest_int("min_child_weight", 1, 5),
        "reg_alpha": trial.suggest_float("reg_alpha", 0, 1),
        "reg_lambda": trial.suggest_float("reg_lambda", 0, 1)
    }
    # Definición de rangos de optimización por hiperparámetro para Onehotencoder
    params_onehot = {
        "min_frequency": trial.suggest_float("min_frequency", 0.1, 1),
    }
    # Definición de columntransformer para preprocesamiento
    preprocessor = ColumnTransformer([
    ('Numerical_cols', PowerTransformer(), numerical_cols),
    ('Categorical_cols', OneHotEncoder(sparse_output=True, **params_onehot), categorical_cols)])
    # Definicion de pipeline con preprocesamiento y XGBRegressor
    pipeline_to_optimize = Pipeline([
    # Implementación de FuncionTransformer en columna 'date'
        ('Transform_date', FunctionTransformer(transform_date)),
    # Implementación de ColumnTransformer en columnas numericas y categoricas
        ('Preprocessor', preprocessor),
    # Implementación de FunctionTransformer para transformar el output a df
        ('Convert_df', FunctionTransformer(
        lambda X: pd.DataFrame(X, columns=numerical_cols + preprocessor.transformers_[1][1].get_feature_names_out().tolist()))),
    # Implementación de XGBRegressor incluyendo la relacion monotona negativa con el precio
        ('XGBRegressor', XGBRegressor(seed=42,  **params_xgb, verbose=False))
        ])
    # Entrenamiento del pipeline
    pipeline_to_optimize.fit(X_train, Y_train)   
    # Prediccion en particion de validacion
    Y_val_preds = pipeline_to_optimize.predict(X_val)
    # Obtencion de métrica MAE a partir de las predicciones en particion de validacion
    MAE_val = mean_absolute_error(Y_val, Y_val_preds)
    #print(f'MAE en particion de validación (XGBRegressor con optimización de parámetros): {MAE_val}')
    return MAE_val

#  Implementacion -----------------------------------------------------------------------------------------------------------------------------------------------------------------------
study = optuna.create_study(direction="minimize", sampler=optuna.samplers.TPESampler(seed=99))
study.optimize(optimize_function, timeout=300, show_progress_bar=True)

# Obtencion de parámetros optimos, MAE óptimo y cantidad de intentos --------------------------------------------------------------------------------------------------------------------
print('Best parameters:', study.best_params)
print('Best value:', study.best_value)
print('Number_trials:', len(study.trials))
MAE_val_xgbregressor_opt = study.best_value

   0%|          | 00:00/05:00

Best parameters: {'learning_rate': 0.0964255209824671, 'n_estimators': 277, 'max_depth': 7, 'max_leaves': 10, 'min_child_weight': 4, 'reg_alpha': 0.31063815361179714, 'reg_lambda': 0.19404647584629156, 'min_frequency': 0.17869075229323728}
Best value: 6469.700953203591
Number_trials: 66


Hiperparámetros:
- **learning_rate:** Parámetro de XGBRegressor() que corresponde a la tasa de aprendizaje para regular "que tanto contribuye" el árbol en la predicción final, o bien, en que grado como se ajustan los pesos del modelo en cada iteración. El rango de optimización de 0.001 a 0.1 es razonable dado que un valor mayor a 0.1 probablemente no presente mejoras significativas en el rendimiento (ya que alcanza un punto de saturación) y a partir de 0.001 se pueden tener aumentos significativos del rendimiento.

- **n_estimators:** Parámetro de XGBRegressor() que corresponde a la cantidad de árboles del modelo. Un mayor valor puede mejorar el rendimiento del modelo pero aumentar el tiempo de ejecución y el riesgo de sobreajuste. El rango de optimización de 50 a 1000 parece razonable dado que un valor mayor a 1000 aumenta la complejidad del modelo a un nivel donde los recursos podrían no ser suficientes (altos tiempos de ejecución por ejemplo) o llegar  aun punto donde el modelo no presente mejoras significativas en el rendimiento (punto de saturación).

- **max_depth:** Parámetro de XGBRegressor() asociado a la complejidad del árbol. Corresponde a la altura máxima (profundidad) del árbol. Un mayor valor puede mejorar el rendimiento (al capturar mas información) pero aumentar el riesgo de sobreajuste por la complejidad del árbol.  El rango de optimización de 3 a 10 parece razonable ya que un valor mayor a 10 aumentaría la complejidad del modelo de forma análoga a n_Estimators.

- **max_leaves:** Parámetro de XGBRegressor() asociado a la complejidad del árbol. Corresponde a la cantidad de nodos terminales (hojas) del árbol. Un mayor valor tiene efectos análogos a un mayor valor en max_depth dado que tambien es un parámetro asociado a la complejidad del árbol. El rango de optimización de 0 a 100 podría tener un mayor valor mínimo de 1 ya que no tiene sentido que un nodo no tenga hojas, además, un nodo con mas de 50 hojas podría aumentar el riesgo de sobreajuste (aunque depende de la complejidad del dataset).

- **min_child_weight:** Parámetro de XGBRegressor() asociado a la complejidad del árbol. Corresponde a la suma define la suma mínima de pesos de instancia que deben estar presentes en un nodo secundario en cada árbol (evitando división de nodos si el peso de las instancias es menor a este valor). Un mayor valor puede disminuir el riesgo de sobreajuste al "crear divisiones de nodos" solo si los datos lo justifican lo suficiente. El rango de optimización de 1 a 5 parece razonable ya que asi se se tiene suficiente flexibilidad para la cantidad de instancias sin comprometer un sobreajuste del modelo. 

- **reg_alpha:** Parámetro de XGBRegressor() asociado a la regularización L1. Ayuda a evitar el sobreajuste penalizando los pesos grandes. Un valor más alto da lugar a un modelo más regularizado. El rango de optimización de 0 a 1 parece un rango de exploración válido para la optimzación de parámetros dado que va desde un modelo con nula regularización a un modelo con mayor regularización (apto para modelos complejos dada la gran cantidad de features), asi se permite simplificar el modelo evitando el sobreajuste considerando que valores mayores a 1 podrían resultar en una regularización excesiva y baja capacidad de generalización del modelo. 

- **reg_lambda:** Parámetro de XGBRegressor() asociado a la regularización L2. Ayuda a evitar el sobreajuste penalizando los pesos grandes de forma cuadrática.  Un valor más alto da lugar a un modelo más regularizado ayudando a evitar el sobreajuste. El rango de optimización de 0 a 1 parece un rango de exploración válido para la optimzación de parámetros por los motivos análogos a reg_alpha.

- **min_frequency:** Parámetro de OneHotEconder(), corresponde a la frecuencia mínima de una categoria para que esta sea codificada. Un valor más alto disminuye la complejidad de la codificación y permite manejar categorias poco frecuentes en el dataset. El rango de optimización de 0 a 1 podría tener un valor mínimo de 0.05 dado que no tiene sentido fijar el parámetro menor a 0.05, esto sería codificar categorias de variables con una frecuencia menor al 5%.

En relación a los resultados obtenidos se obtuvo un MAE de 6547, menor al mejor rendimiento que se tenia (XGBRegressor sin optimización de parámetros y sin restricción de relación con un MAE de 7012). El resultado anterior se obtuvo despues de 165 iteraciones de ajuste de hiperparámetros considerando un tiempo límite de 5 minutos. En este caso, podemos notar que la optimización de parámetros si implico una mejora considerable con respecto al modelo del baseline. En particular, se reportaron los siguientes parámetros óptimos:

-   **learning_rate':** 0.08855968023444559
-   **'n_estimators':** 734
-   **'max_depth':** 4
-   **'max_leaves':** 6
-   **'min_child_weight':** 4
-   **'reg_alpha':** 0.10242482571077649
-   **'reg_lambda':** 0.017327249149241303
-   **'min_frequency':** 0.14359042482925244

Podemos asociar esta mejora del rendimiento del modelo a la inclusión de la optimización de parámetros considerando que se itera en múltiples grillas de combinaciones de estos buscando minimizar el MAE (optimizar el rendimiento), por lo anterior, a medida que se aumente el número de iteraciones del modelo el MAE mínimo debería ir disminuyendo. Sin embargo, cabe destacar que si se "optimiza demasiado" el rendimiento del modelo en función del ajuste de estos hiperparámetros, el modelo tiene mayor riesgo de sobreajuste.

## 4. Optimización de Hiperparámetros con Optuna y Prunners (1.7)

<p align="center">
  <img src="https://i.pinimg.com/originals/90/16/f9/9016f919c2259f3d0e8fe465049638a7.gif">
</p>

Después de optimizar el rendimiento de su modelo varias veces, Fiu le pregunta si no es posible optimizar el entrenamiento del modelo en sí mismo. Después de leer un par de post de personas de dudosa reputación en la *deepweb*, usted llega a la conclusión que puede cumplir este objetivo mediante la implementación de **Prunning**.

Vuelva a optimizar los mismos hiperparámetros que la sección pasada, pero esta vez utilizando **Prunning** en la optimización. En particular, usted debe:

- Responder: ¿Qué es prunning? ¿De qué forma debería impactar en el entrenamiento?
- Utilizar `optuna.integration.XGBoostPruningCallback` como método de **Prunning**
- Fijar nuevamente el tiempo de entrenamiento a 5 minutos
- Reportar el número de *trials*, el `MAE` y los mejores hiperparámetros encontrados. ¿Cómo cambian sus resultados con respecto a la sección anterior? ¿A qué se puede deber esto?
- Guardar su modelo en un archivo .pkl

Nota: Si quieren silenciar los prints obtenidos en el prunning, pueden hacerlo mediante el siguiente comando:

```
optuna.logging.set_verbosity(optuna.logging.WARNING)
```

De implementar la opción anterior, pueden especificar `show_progress_bar = True` en el método `optimize` para *más sabor*.

Hint: Si quieren especificar parámetros del método .fit() del modelo a través del pipeline, pueden hacerlo por medio de la siguiente sintaxis: `pipeline.fit(stepmodelo__parametro = valor)`

Hint2: Este <a href = https://stackoverflow.com/questions/40329576/sklearn-pass-fit-parameters-to-xgboost-in-pipeline>enlace</a> les puede ser de ayuda en su implementación

In [8]:
# Funcion para optimizar parametros del modelo con prunning --------------------------------------------------------------------------------------------------------------------------------------------
def optimize_function_prunning(trial):
    params_xgb = {
        "learning_rate": trial.suggest_float("learning_rate", 0.001, 0.1),
        "n_estimators": trial.suggest_int("n_estimators", 50, 1000),
        "max_depth": trial.suggest_int("max_depth", 3, 10),
        "max_leaves": trial.suggest_int("max_leaves", 0, 100),
        "min_child_weight": trial.suggest_int("min_child_weight", 1, 5),
        "reg_alpha": trial.suggest_float("reg_alpha", 0, 1),
        "reg_lambda": trial.suggest_float("reg_lambda", 0, 1)
    }
    params_onehot = {
        "min_frequency": trial.suggest_float("min_frequency", 0.1, 1),
    }
    preprocessor = ColumnTransformer([
    ('Numerical_cols', PowerTransformer(), numerical_cols),
    ('Categorical_cols', OneHotEncoder(sparse_output=True, **params_onehot), categorical_cols)
])
    pipeline = Pipeline([
    # Implementación de FuncionTransformer en columna 'date'
        ('Transform_date', FunctionTransformer(transform_date)),
    # Implementación de ColumnTransformer en columnas numericas y categoricas
        ('Preprocessor', preprocessor),
    # Implementación de FunctionTransformer para transformar el output a df
        ('Convert_df', FunctionTransformer(
        lambda X: pd.DataFrame(X, columns=numerical_cols + preprocessor.transformers_[1][1].get_feature_names_out().tolist())))
        ])
    pruning_callback = optuna.integration.XGBoostPruningCallback(trial, "validation_0-mae")

    # Entrenar el preprocesador
    pipeline.fit(X_train)

    X_train_transformed = pipeline.transform(X_train)
    X_val_transformed = pipeline.transform(X_val)
    
    # Definir el modelo como un pipeline separado
    model = XGBRegressor(seed=42,eval_metric='mae', **params_xgb)

    # Entrenar el modelo
    model.fit(X_train_transformed, Y_train, callbacks=[pruning_callback], eval_set=[(X_train_transformed, Y_train), (X_val_transformed, Y_val)], verbose=False)

    # Predict y evaluar el modelo
    X_val_transformed = pipeline.transform(X_val)
    Y_val_preds = model.predict(X_val_transformed)

    # Calculo de MAE
    MAE_val = mean_absolute_error(Y_val, Y_val_preds)
    #print(f'MAE en particion de validación (XGBRegressor con optimización de parámetros y prunning): {MAE_val}')
    return MAE_val


#  Implementacion -----------------------------------------------------------------------------------------------------------------------------------------------------------------------
study_prunning = optuna.create_study(direction="minimize", sampler=optuna.samplers.TPESampler(seed=42), )
study_prunning.optimize(optimize_function_prunning, timeout=300, show_progress_bar=True)


# Obtencion de parámetros optimos, MAE óptimo y cantidad de intentos --------------------------------------------------------------------------------------------------------------------
print('Best parameters:', study_prunning.best_params)
print('Best value:', study_prunning.best_value)
print('Number_trials:', len(study_prunning.trials))
MAE_val_xgbregressor_opt_prunning = study_prunning.best_value

   0%|          | 00:00/05:00

Best parameters: {'learning_rate': 0.08778352201499837, 'n_estimators': 616, 'max_depth': 3, 'max_leaves': 100, 'min_child_weight': 3, 'reg_alpha': 0.7882937674472155, 'reg_lambda': 0.5173525652140668, 'min_frequency': 0.1622591111294117}
Best value: 6512.389083748602
Number_trials: 74



El "pruning" ("poda" en español) es una técnica enfocada en mejorar la eficiencia de algoritmos de optimización, especialmente en el contexto del ajuste de hiperparámetros de modelos. Consiste en eliminar "tempranamente" aquellas iteraciones en las que las configuraciones de hiperparámetros tengan pocas probabilidades de producir mejores resultados en función del historial de resultados en las iteraciones anteriores. Esto permite concentrar los recursos computacionales en las configuraciones más prometedoras.

En este caso, se implementa prunning sobre la optimización de parámetros de optuna sobre el XGBRegressor. En particular, notamos inmediatamente que la velocidad promedio de entrenamiento por iteración tiende a ser mucho menor  y además, es de esperar que se obtengan menores MAE "mínimos" (óptimos) dado que puede realizar mayor cantidad de iteraciones en menor tiempo y que estas tienden a retornar un mejor rendimiento del modelo con el ajuste de sus hiperparámetros. Por lo tanto, el prunning en el entrenamiento reduce los tiempos de ejecución, acelera la convergencia al conjunto de parámetros óptimos y deberia retorna un MAE menor (despues vemos que no es así pipipi).

Notamos que en comparación con el mismo modelo sin prunning, se tiene una mayor cantidad de iteraciones y un menor MAE tal como se esperaba.


## 5. Visualizaciones (0.5 puntos)

<p align="center">
  <img src="https://media.tenor.com/F-LgB1xTebEAAAAd/look-at-this-graph-nickelback.gif">
</p>


Satisfecho con su trabajo, Fiu le pregunta si es posible generar visualizaciones que permitan entender el entrenamiento de su modelo.

A partir del siguiente <a href = https://optuna.readthedocs.io/en/stable/tutorial/10_key_features/005_visualization.html#visualization>enlace</a>, genere las siguientes visualizaciones:

1. Gráfico de historial de optimización
2. Gráfico de coordenadas paralelas
3. Gráfico de importancia de hiperparámetros

Comente sus resultados:

4. ¿Desde qué *trial* se empiezan a observar mejoras notables en sus resultados?
5. ¿Qué tendencias puede observar a partir del gráfico de coordenadas paralelas?
6. ¿Cuáles son los hiperparámetros con mayor importancia para la optimización de su modelo?

In [9]:
# 1. Grafico de historial de optimización ------------------------------------------------------
plot_optimization_history(study_prunning)

In [10]:
# 2. Gráfico de coordenadas paralelas ------------------------------------------------------
plot_parallel_coordinate(study_prunning)

In [11]:
# 3. Gráfico de importancia de hiperparámetros
plot_param_importances(study_prunning)

A partir del gráfico de historial de optimización notamos que el valor best_value correspondiente al mejor mínimo del MAE registrado es actualizado en 3 trials especificos de forma significativa, el primero inmediatamente luego de la primera iteración donde se actualiza a 7250, el segundo en la iteración 13 donde se actualiza a 6601.

A partir del gráfico de coordenadas paralelas permite notar ciertas "cotas" dentro del rango de optimización explorado, en particular, aquellos valores que presentan una concentración mayor de enlaces, presenta un rendimiento "mejor" del modelo en relación a la minimización del MAE. Con lo anterior, se podría ir reduciendo el rango de optimización de los paramétros gracias a estas cotas, "acercandonos" cada vez mas a la convergencia del óptimo y aprovechando los recursos en iteraciones con grillas de parámetros que son mas probables de tener un mejor rendimiento del modelo final.

Entre algunas observaciones, se tiene que:
-  El rango de learning_rate estaría acotado por 0.08 y 0.09 y no entre 0.001 y 0.1 como se indico inicialmente como rango de exploración.
-  El rango de max_depth no se aprecia acotado significativamente por lo que el rango de exploración inicial sería adecuado.
-  El rango de max_leaves tiende a estar acotado entre 50 y 100 y no 0 y 100 como se indico inicialmente como rango de exploración.
-  El rango de min_child_weight esta acotado entre 1 y 3 y no entre 1 y 5 como se indico inicialmente como rango de exploración.
-   El rango de min_frequency estaría acotado por 0.1 y 0.2 y no entre 0.1 y 1 como se indico inicialmente como rango de exploración.
-   El rango de n_estimators estaría acotado por 50 y 800 y no entre 50 y 1000 como se indico inicialmente como rango de exploración.
- El rango de reg_alpha estaría acotado por 0.8 y 0.9 y no 0 y 1 como se indico inicialmente en el rango de exploración.
- El rango de reg_lambda no se aprecia acotado significativamente por lo que el rango de exploración inicial sería adecuado.

A partir del gráfico de importancia de hiperparámetros notamos que el parámetro de min_frequency es aquel que tiene una importancia relativa cercana al 68% en el rendimiento del modelo seguido por el parámetro de learning_rate con una importancia relativa de 22% aproximadamente. Por lo tanto, la optimización de estos dos parámetros principales es crucial para optimizar el valor del MAE. 

## 6. Síntesis de resultados (0.3)

Finalmente:

1. Genere una tabla resumen del MAE obtenido en los 5 modelos entrenados desde Baseline hasta XGBoost con Constraints, Optuna y Prunning.
2. Compare los resultados de la tabla y responda, ¿qué modelo obtiene el mejor rendimiento?
3. Cargue el mejor modelo, prediga sobre el conjunto de **test** y reporte su MAE.
4. ¿Existen diferencias con respecto a las métricas obtenidas en el conjunto de validación? ¿Porqué puede ocurrir esto?

In [12]:
# df resultados
resultados = pd.DataFrame({"Modelo":["DummyRegressor:","XGBRegressor","XGBRegressor con restriccion","XGBRegressor con optuna","XGBRegressor con optuna y prunning"],
                            "MAE Validación":[MAE_val_dummyregressor, MAE_val_xgbregressor, MAE_val_xgbregressor_ct, MAE_val_xgbregressor_opt, MAE_val_xgbregressor_opt_prunning]})

resultados

,Modelo,MAE Validación
0,DummyRegressor:,13576.566991
1,XGBRegressor,7012.852411
2,XGBRegressor con restriccion,7367.636682
3,XGBRegressor con optuna,6469.700953
4,XGBRegressor con optuna y prunning,6512.389084


A partir de lo anterior, notamos que el modelo con mejor rendimiento es el modelo con XGBRegressor utilizando optimización de parámetros (Sin prunning) con un MAE de 6464.



In [13]:
# Definición de columntransformer para preprocesamiento
preprocessor = ColumnTransformer([
    ('Numerical_cols', PowerTransformer(), numerical_cols),
    ('Categorical_cols', OneHotEncoder(sparse_output=True), categorical_cols)])

# Definicion de pipeline con preprocesamiento y XGBRegressor
pipeline_to_optimize = Pipeline([
    # Implementación de FuncionTransformer en columna 'date'
    ('Transform_date', FunctionTransformer(transform_date)),
    # Implementación de ColumnTransformer en columnas numericas y categoricas
    ('Preprocessor', preprocessor),
    # Implementación de FunctionTransformer para transformar el output a df
    ('Convert_df', FunctionTransformer(
        lambda X: pd.DataFrame(X, columns=numerical_cols + preprocessor.transformers_[1][1].get_feature_names_out().tolist())))
])

# Entrenar el preprocesador con todos los datos de entrenamiento
pipeline_to_optimize.fit(X_train.append(X_val))

# Transformar X_train y X_test
X_train_transformed = pipeline_to_optimize.transform(X_train)
X_test_transformed = pipeline_to_optimize.transform(X_test)

# Entrenar el modelo con todos los datos de entrenamiento
model = XGBRegressor(seed=42, eval_metric='mae', **study.best_params)
model.fit(X_train_transformed, Y_train)

# Realizar predicciones en X_test
Y_test_preds = model.predict(X_test_transformed)

# Calcular MAE en X_test
MAE_test = mean_absolute_error(Y_test, Y_test_preds)
print(f'MAE en partición de test (XGBRegressor con optimización de parámetros): {MAE_test}')



MAE en partición de test (XGBRegressor con optimización de parámetros): 6688.591937599489


Con lo anterior, se tiene que en la particion de testeo se obtuvo un MAE de 6764 bastante cercano al valor reportado en la particion de validación. Esta diferencia podría deberse a que los datos de validación no son representativos de los datos en testeo, ya sea por baja cantidad de datos o por distribuciones muy dispersas o directamente distintas.

# Conclusión
Eso ha sido todo para el lab de hoy, recuerden que el laboratorio tiene un plazo de entrega de una semana. Cualquier duda del laboratorio, no duden en contactarnos por mail o U-cursos.

<p align="center">
  <img src="https://media.tenor.com/8CT1AXElF_cAAAAC/gojo-satoru.gif">
</p>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>